<a href="https://colab.research.google.com/github/mgkim-developer/INU-Lab/blob/master/ECG_analysis_LSTM_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras. layers import Dense, Activation, Dropout
from keras.layers import LSTM
import numpy as np
from keras.callbacks import EarlyStopping
import pandas as pd
from sklearn import preprocessing, datasets
from sklearn.metrics import mean_squared_error
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten



np.random.seed(3)

df = pd.read_csv('/content/drive/My Drive/Colab_Data/heartbeat_datasets_edit2.csv')
df.isnull().sum()
print(df.values.shape)

dataset = df.values
print(dataset)

x = dataset[:, 0:-1]
y = dataset[:, -1]

x_train = x[:189]
y_train = y[:189]

x_val = x[189:243]
y_val = y[189:243]

x_test = x[243:]
y_test = y[243:]

print(x_test[:10])



# #정규화

x_train = x_train.reshape(189, 986).astype('float32')/1620.0
x_val = x_val.reshape(54, 986).astype('float32')/1620.0
x_test = x_test.reshape(27, 986).astype('float32')/1620.0

#data handling
x_train = np.reshape(x_train, (189, 986, 1))
x_val = np.reshape(x_val, (54, 986, 1))
x_test = np.reshape(x_test, (27, 986, 1))

# x_train = np.reshape(x_train, (x_train.shape[0[0], x_train.shape[1],))
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],3))

#data 구성
model = Sequential()
model.add( LSTM(32, input_shape = (986,1)))
model.add(Dropout(0.2))
model.add( Dense(64, activation = 'relu'))
model.add(Dropout(0.2))
model.add( Dense(32, activation = 'relu'))
model.add( Dense(16, activation = 'relu'))
model.add( Dense(3, activation = 'softmax'))




Using TensorFlow backend.


(270, 986)
[[970 950 970 ... 950 950 'Baseline']
 [760 740 750 ... 240 240 'VT']
 [940 780 780 ... 700 700 'Baseline']
 ...
 [490 1140 820 ... 270 280 'VT']
 [1070 1040 1050 ... 1200 1170 'Baseline']
 [740 590 920 ... 320 290 'VT']]
[[610 580 590 ... 300 320 290]
 [780 810 800 ... 790 800 790]
 [750 740 750 ... 360 320 340]
 ...
 [940 960 950 ... 1080 1070 1030]
 [580 580 590 ... 350 360 360]
 [740 760 580 ... 1030 1050 980]]


ValueError: ignored

In [10]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
import numpy as np
from keras.callbacks import EarlyStopping 
import pandas as pd
from sklearn import preprocessing, datasets 
from sklearn.metrics import mean_squared_error

np.random.seed(3)

df = pd.read_csv('/content/drive/My Drive/Colab_Data/heartbeat_datasets.csv')
print(df.values.shape)

dataset = df.values
print(dataset)

X = dataset[:, 0:-1]
y = dataset[:, -1]

x_train = X[:189]
y_train = y[:189]

x_val = X[189:243]
y_val = y[189:243]

x_test = X[243:]
y_test = y[243:]

print(x_test[:10])

# 2. 정규화
x_train = x_train.reshape(189, 986).astype('float32')/1620.0
x_val = x_val.reshape(54, 986).astype('float32')/1620.0
x_test = x_test.reshape(27, 986).astype('float32')/1620.0

# print(x_train.shape)
x_train = np.reshape(x_train, (189, 986, 1))
x_val = np.reshape(x_val, (54, 986, 1))
x_test = np.reshape(x_test, (27, 986, 1))

# x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],))
# x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],3))

# 3. 모델 구성
model = Sequential()
model.add( LSTM(32, input_shape = (986, 1)))
model.add(Dropout(0.2)) 
model.add( Dense(64, activation = 'relu')) 
model.add(Dropout(0.2)) 
model.add( Dense(32, activation = 'relu'))
model.add( Dense(16, activation = 'relu'))
model.add( Dense(3, activation = 'softmax'))

# 4. model compile
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# 5. 학습의 자동중단 10번동안 나아지지 않으면 stop
early_stop = EarlyStopping(monitor='val_loss', patience=15)

# 6. 모델 학습
hist = model.fit(x_train, y_train, epochs=500, batch_size=32, validation_data=(x_val, y_val),callbacks=[early_stop])

# 7. 모델 학습 과정 표시하기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

# 8. 결과

loss_and_metrics = model.evaluate(x_test, y_test, batch_size=1)
print('loss : ' + str(loss_and_metrics[0])) 
results = model.evaluate(x_test, y_test)
print('acc:', results[1]*100,'%')

predicts= model.predict(x_test)
# 9. 예측한 값 분류화 
result = model.predict_classes(x_test, verbose=0)
print("예측값 softmax 적용\n", predicts[:10] )
print("\n클래스예측\n", result[:10])
print("\n실제값\n", y_test[:10])
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True, dpi=60).create(prog='dot', format='svg'))

SyntaxError: ignored